In [18]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.document_loaders import UnstructuredFileLoader, DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Qdrant
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient

In [19]:
load_dotenv()
client = QdrantClient(
    url=os.getenv("Qdrant_HOST"), 
    api_key=os.getenv("Qdrant_API_KEY")
)

In [25]:
from qdrant_client.http import models
load_dotenv()
vector_config = models.VectorParams(
    size=768,
    distance=models.Distance.COSINE
)

client.recreate_collection(
    collection_name=os.getenv("Qdrant_Collection_name"),
    vectors_config=vector_config
)

True

In [26]:
load_dotenv()
embeddings=SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

vector_store = Qdrant(
    client=client,
    collection_name=os.getenv("Qdrant_Collection_name"),
    embeddings=embeddings
)


In [27]:
def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator='\n',
        chunk_size=1200,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

text_directory = './data2'

# Read each text file and apply the get_chunks function
all_chunks = []
for text_file in os.listdir(text_directory):
    if text_file.endswith('.txt'):
        file_path = os.path.join(text_directory, text_file)
        with open(file_path, 'r', encoding='utf-8') as file:
            text_content = file.read()
            file_chunks = get_chunks(text_content)
            all_chunks.extend(file_chunks)

vector_store.add_texts(all_chunks)

ResponseHandlingException: The write operation timed out